In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

# 1. 取出csv数据，按照MMSI和Timestamp排列，将时间戳转化为时间间距，保存为.npz

In [2]:
# enter path here
data_path = r''
save_path =  r''
save_path_train =  r''
save_path_predict =  r''

In [24]:
fields = [0, 2, 8, 7, 5, 9 ] # Timestamp, MMSI, Lat, Long, SOG, COG
# n_rows = 30000              # Pulls this many rows of data, because all of it is too much

In [30]:
df = pd.read_csv(data_path,encoding='gbk',usecols=fields)

In [31]:
df = df.dropna()

In [29]:
df.head()

,时间,船舶MMSI,SOG,经度,纬度,COG
0,2023-01-30 23:59:59,413376990,9.5,122.304143,31.102057,123.9
1,2023-01-31 00:00:00,414574000,10.7,120.617920,38.629153,110.8
2,2023-01-31 00:00:00,413324940,0.1,119.639500,37.800067,290.1
3,2023-01-31 00:00:00,525125006,16.5,121.019968,38.483102,119.5
4,2023-01-31 00:00:00,413436370,0.0,121.958908,38.825000,251.6


In [32]:
df = df[['时间','船舶MMSI','纬度','经度','SOG','COG']]

In [33]:
# change dataframe to numpy array
df = df.values

# new number of rows and columns
n_rows, n_cols = df.shape

In [35]:
# sort by MMSI, then by time/date
df = df[np.lexsort((df[:, 0], df[:, 1]))]

In [36]:
df

array([['2023-01-31 00:00:37', 0, 38.32315333333333, 121.00849666666667,
        0.0, 0.0],
       ['2023-01-31 00:00:58', 0, 38.32346666666667, 120.9729, 0.0, 0.0],
       ['2023-01-31 00:01:02', 0, 24.33272833333333, 118.19321, 0.7,
        59.0],
       ...,
       ['2023-01-31 23:12:01', 1020001801, 37.595845, 121.40482, 0.0,
        303.5],
       ['2023-01-31 23:25:21', 1020001801, 37.59581833333333,
        121.40480166666669, 0.0, 303.5],
       ['2023-01-31 23:54:53', 1020001801, 37.59055833333333,
        121.40360333333334, 6.3, 188.6]], dtype=object)

In [37]:
# convert time to int (unit:second)
for i in range(n_rows):
    df[i][0] = int(df[i][0][11:13])*3600 + int(df[i][0][14:16])*60 + int(df[i][0][17:])

In [38]:
# create timedeltas
i = 0
while i in range(10):
    end = False
    temp = []
    start = i
    try:
        while df[i+1][1] == df[i][1]:
            temp.append(df[i][0])
            i += 1
            end = True
    except: pass

    if end is True:
        temp.append(df[i][0])
        diff_array = np.diff(temp)

        df[start][0] = 0   # inital point 
        df[start+1:i+1, 0] = diff_array  # difference 
    i += 1

In [39]:
np.savez(save_path, sorted_data=df)

# 2. 将数据转化为数据集
- 数据标准化
- 舍弃同一个MMSI少于5个数据点的数据
- 将5个连续时间点作为输入，第6给作为输出
- input = (batch_size, 5 , 5(time, lat,long, sog.cog))
- output = (batch_size, 1, 2(lat,long))

In [3]:
# read data
sorted_data = np.load(save_path,allow_pickle=True)
df = sorted_data['sorted_data']

In [4]:
# take out mmsi column
second_column = df[:, 1]
# find number of batches (Those with less than 5 data points for the same MMSI will be discarded)
batch_count = 0
unique_vals, unique_count = np.unique(second_column, return_counts=True)
for i in unique_count:
    if i >= 5:
        batch_count += (i - 4)

In [5]:
# create empty x,y train
x_train = np.empty([batch_count, 5, 6])
y_train = np.empty([batch_count, 6])

In [6]:
first_lat = []   # first lat of each MMSI
first_long = []  # first long of each MMSI
k = 0
# make data set into relative latitudes and longitudes
for i in range(len(unique_count)):
    first_lat.append(df[k][2])
    first_long.append(df[k][3])
    k += unique_count[i]

k = 0
last_count = 0
for i in range(len(df)):
    df[i, 2] = df[i, 2] - first_lat[k]
    df[i, 3] = df[i, 3] - first_long[k]
    if (i - last_count) == unique_count[k]:
        last_count = i
        k += 1

In [44]:
# normalize dataset
lat_min = -90
lat_max = 90
long_min = -180
long_max = 180
speed_min = 0
speed_max = np.amax(df[:, [4]])
time_min = 0
time_max = np.amax(df[:, [0]])
course_min = 0
course_max = np.max(df[:, [5]])

# change data from 0 to 1
# df[:, [2]] = ((df[:, [2]] - lat_min)/(lat_max - lat_min))
# df[:, [3]] = ((df[:, [3]] - long_min)/(long_max - long_min))
df[:, [0]] = (df[:, [0]] - time_min) / (time_max - time_min)
df[:, [4]] = (df[:, [4]] - speed_min) / (speed_max - speed_min)
df[:, [5]] = (df[:, [5]] - course_min) / (course_max - course_min)

In [ ]:
# fill batches
i = 0

# 5 timestep as input and 1 timestep as output
for count, k in enumerate(second_column):
    try:
        if second_column[count + 5] == k:
            x_train[i][0][:] = df[count][:]
            x_train[i][1][:] = df[count + 1][:]
            x_train[i][2][:] = df[count + 2][:]
            x_train[i][3][:] = df[count + 3][:]
            x_train[i][4][:] = df[count + 4][:]

            y_train[i][:] = df[count + 5][:]
            i += 1
    except:
        pass

In [ ]:
# take out name feature column, cut to size of i above
x_train = x_train[:i - 1, :, [0, 2, 3, 4, 5]]
y_train = y_train[:i - 1, [2, 3]]

# splice data for training and testing
train_length = int(i * 0.8)
y_test = y_train[train_length:, :]
x_test = x_train[train_length:, :, :]

y_train = y_train[:train_length + 1, :]
x_train = x_train[:train_length + 1, :, :]

In [ ]:
np.savez(save_path_train, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

# *可视化

In [3]:
sorted_data = np.load(save_path,allow_pickle=True)
df = sorted_data['sorted_data']

test = []
temp = []

In [4]:
# take out MMSI column
second_column = df[:, 1]

In [5]:
unique_vals, unique_count = np.unique(second_column, return_counts=True)

In [6]:
k = 0
last_count = 0
for i in range(len(df)):
    temp.append(tuple([df[i][2], df[i][3]]))
    if (i - last_count) == unique_count[k]:
        test.append(temp)
        temp = []
        last_count = i
        k += 1
test.append(temp)

In [7]:
center = [df[i-1][2], df[i-1][3]]

In [22]:
m = folium.Map(location=center, tiles="OpenStreetMap", zoom_start=6)

In [23]:
for i in range(len(test[600:700])):
    for j in range(len(test[i])):
        folium.Circle(
            radius=2,
            location = test[i][j],
        ).add_to(m)

In [26]:
m.save("vis-2022-11-01.html")

# 3.模型预测

In [7]:
import glob
import os
from math import sin, cos, sqrt, atan2, radians
from tensorflow.keras.models import model_from_json
from tensorflow.keras import optimizers
from tensorflow.keras import backend
from sklearn.metrics import mean_squared_error

In [8]:
model_name_json = r''        #  enter json path here
model_name_h5 = r''          # enter h5 path here

In [9]:
training_data = np.load(save_path_train)
x_test = training_data['x_test']
y_test = training_data['y_test']

In [10]:
# Finding the most recent files
list_of_jsons = glob.glob('../../models/*.json')
list_of_jsons.sort(key=os.path.getctime, reverse=True)

list_of_h5 = glob.glob('../../models/*.h5')
list_of_h5.sort(key=os.path.getctime, reverse=True)

In [11]:
json_file = open(model_name_json, 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(model_name_h5)

In [13]:
def firsts():
    lats = first_lat
    longs = first_long
    unique = unique_count
    return [lats, longs, unique]

In [14]:
# Function Definitions
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))


# Get distance between pairs of lat-lon points (in meters)
def distance(lat1, lon1, lat2, lon2):
    r = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    dist = r*c*1000

    return dist


# Custom adam optimizer
adam = optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# evaluate loaded model on test data
loaded_model.compile(loss='mse',
                     optimizer=adam,
                     metrics=[rmse])

# Predict Outputs
prediction = loaded_model.predict(x_test)

In [15]:
mean_squared_error(y_test,prediction)

48.14887823197202

In [16]:
# Post Processing
firsts = firsts()
first_lat = firsts[0]
first_long = firsts[1]
unique_count = firsts[2]

# Adding lats and longs back to give actual predictions
k = 0
last_count = 0
for i in range(len(y_test)):
    prediction[i, 0] = prediction[i, 0] + first_lat[k]
    prediction[i, 1] = prediction[i, 1] + first_long[k]
    y_test[i, 0] = y_test[i, 0] + first_lat[k]
    y_test[i, 1] = y_test[i, 1] + first_long[k]
    if (last_count - i) == unique_count[k]:
        last_count = i
        k += 1

In [17]:
# Determining average distance between prediction and y_test
df_lls = pd.DataFrame({'lat1': prediction[:, 0], 'lon1': prediction[:, 1], 'lat2': y_test[:, 0], 'lon2': y_test[:, 1]})

dist = np.empty(len(df_lls['lat1']))

for i in range(dist.size):
    dist[i] = distance(df_lls['lat1'][i],
                          df_lls['lon1'][i],
                          df_lls['lat2'][i],
                          df_lls['lon2'][i])

# Find the average distance in meters
#  avg_dist = np.mean(dist) Maybe work
nine_sort = np.sort(dist)
avg_dist = nine_sort[int(0.9*len(dist))]  # currently the bottom 90% distances of sorted distances

In [18]:
y_test.shape

(1930570, 2)

In [19]:
np.savez(save_path_predict, prediction=prediction, y_test=y_test)

In [20]:
print('----------------------------------------------------')
print('Average Distance (m): ', avg_dist, ' m')
print('Average Distance (km): ', avg_dist / 1000, ' km')
print('Average Distance (NM): ', avg_dist * 0.00053996, 'NM')
print('----------------------------------------------------')
print('End of danish_predict.py')
print('----------------------------------------------------')

----------------------------------------------------
Average Distance (m):  6777.834379182011  m
Average Distance (km):  6.777834379182011  km
Average Distance (NM):  3.6597594513831186 NM
----------------------------------------------------
End of danish_predict.py
----------------------------------------------------


In [21]:
# graph AOU

location_index = 50

center = [prediction[location_index, 0], prediction[location_index, 1]]


In [34]:
m = folium.Map(
        location=center,
        zoom_start=6,
        tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=7&x={x}&y={y}&z={z}', # 高德街道图
#         tiles='http://webst02.is.autonavi.com/appmaptile?style=6&x={x}&y={y}&z={z}', # 高德卫星图
        attr='default')

In [35]:
prediction.shape

(1930570, 2)

In [36]:
size, index = prediction.shape
for i in range(70000):
    folium.Circle(
        radius=2,
        location=[y_test[i, 0], y_test[i, 1]],
        popup='Real Location',
        color='crimson',
        fill=True,
        fill_color='#ffcccb'
    ).add_to(m)

    # AOU
    folium.Circle(
        location=[prediction[i, 0], prediction[i, 1]],
        radius=2,  # might want to take largest distance!
        popup='AOU Radius: ' + str(avg_dist) + ' meters',
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(m)

m.save("predict_aohai2.html")